In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import utils.functions as f

c:\Users\Lorenzo\miniconda3\lib\site-packages\torch\_functorch\deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
dataset = "direct_matching_20240213"  # "direct_matching_20240213" or "reverse_matching_20240213"
df_processed = pd.read_csv(f"Data/{dataset}_processed.csv", sep=";")

In [23]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric

df = df_processed.copy()
sector_column = "job_sector"
sector = 1
protected_attribute = "cand_gender"


# Pre-filter DataFrame for the sector
sector_df = df[df[sector_column] == sector]

# Retrieve unique job IDs for the specified sector
job_list = sector_df["job_id"].unique()

job = job_list[0]
job_df = sector_df.copy()
job_df["idoneous"] = (job_df["job_id"] == job).astype(int)

# Drop unnecessary columns
job_df = job_df.drop(columns=["job_id", "job_professional_category"])

# Prepare dataset for AIF360 analysis
binaryLabelDataset = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=job_df,
    label_names=["idoneous"],
    protected_attribute_names=[protected_attribute],
)

# Calculate metrics
metric_orig = BinaryLabelDatasetMetric(
    binaryLabelDataset,
    privileged_groups=[{protected_attribute: 1}],
    unprivileged_groups=[{protected_attribute: 0}],
)

original_di = metric_orig.disparate_impact()
original_sdp = metric_orig.statistical_parity_difference()
print(f"Disparate Impact: {original_di}")
print(f" Statistical Parity Difference: {original_sdp}")

Disparate Impact: 1.4794520547945205
 Statistical Parity Difference: 0.035514967021816335


In [21]:
from aif360.algorithms.preprocessing import DisparateImpactRemover

level = 0.8
di = DisparateImpactRemover(repair_level=level)

# Apply the repair algorithm
binaryLabelDataset_repaired = di.fit_transform(binaryLabelDataset)

# Calculate metrics
metric_repaired = BinaryLabelDatasetMetric(
    binaryLabelDataset_repaired,
    privileged_groups=[{protected_attribute: 1}],
    unprivileged_groups=[{protected_attribute: 0}],
)

repaired_di = metric_repaired.disparate_impact()
repaired_sdp = metric_repaired.statistical_parity_difference()
print(f"Disparate Impact: {repaired_di}")
print(f" Statistical Parity Difference: {repaired_sdp}")

print(f"Disparate Impact Change: {original_di - repaired_di}")
print(f" Statistical Parity Difference Change: {original_sdp - repaired_sdp}")
# Ovviamente non cambia nulla, perché non c'è nessun ri-allenamento del modello, gli idonei sono sempre gli stessi

Disparate Impact: 1.4794520547945205
 Statistical Parity Difference: 0.035514967021816335
Disparate Impact Change: 0.0
 Statistical Parity Difference Change: 0.0


In [31]:
# Compare the original and repaired datasets
job_df_orig = binaryLabelDataset.convert_to_dataframe()[0]
job_df_repaired = binaryLabelDataset_repaired.convert_to_dataframe()[0]

In [33]:
# Print the differences between the two datasets
for column in job_df_orig.columns:
    difference = job_df_orig[column] - job_df_repaired[column]
    if difference.sum() != 0:
        print(f"\n{column}")
        print(difference.sum())


cand_id
62535301.0

distance_km
272.019893528

match_score
1.1751099399999276

cand_domicile_province
106.0

cand_domicile_region
22.0

cand_education
7.0

Croato
2.0
